# Problem 2 - Transportation

In [44]:
using CSV, DataFrames

In [45]:
t = CSV.read("/Users/benjaminossyra/Desktop/Optimization/Homework_5/transportation.csv", DataFrame; delim = ',', header=true)

Row,Column1,R1,R2,R3,R4,R5,R6,R7,Supply
,String7,Int64,Int64?,Int64,Int64?,Int64,Int64,Int64,Int64?
1,SA,78,28,22,28,32,164,16,2800
2,SB,54,missing,24,missing,52,190,34,5200
3,SC,48,28,34,26,56,198,40,5800
4,Demand,1800,2400,1200,800,3400,2200,2000,missing


In [46]:
tm = Matrix(t)

4×9 Matrix{Any}:
 "SA"        78    28           22  …    32   164    16  2800
 "SB"        54      missing    24       52   190    34  5200
 "SC"        48    28           34       56   198    40  5800
 "Demand"  1800  2400         1200     3400  2200  2000      missing

Create a vector of the demand values

In [47]:
demand = tm[4,2:8]

7-element Vector{Any}:
 1800
 2400
 1200
  800
 3400
 2200
 2000

Create a vector of the supply values

In [48]:
supply = tm[1:3,9]

3-element Vector{Any}:
 2800
 5200
 5800

We are going to create a vector of the costs, and exclude the missing values since those routes are not possible

In [49]:
c = cat(tm[1,2:8],tm[2,2:8],tm[3,2:8], dims=(1,1))

21-element Vector{Any}:
  78
  28
  22
  28
  32
 164
  16
  54
    missing
  24
    missing
  52
 190
  34
  48
  28
  34
  26
  56
 198
  40

In [50]:
c = collect(skipmissing(c))

19-element Vector{Any}:
  78
  28
  22
  28
  32
 164
  16
  54
  24
  52
 190
  34
  48
  28
  34
  26
  56
 198
  40

Create a vector of the nodes. Both supplier and retailer

In [51]:
nodes = ["SA", "SB", "SC", "R1", "R2", "R3", "R4", "R5", "R6", "R7"]

10-element Vector{String}:
 "SA"
 "SB"
 "SC"
 "R1"
 "R2"
 "R3"
 "R4"
 "R5"
 "R6"
 "R7"

In [52]:
links=[]
for i in 1:3
    for j in 4:10
        if (nodes[i], nodes[j]) != ("SB", "R2") && (nodes[i], nodes[j]) != ("SB", "R4")
            push!(links, (nodes[i], nodes[j]))
        end
    end
end

We have created a vector of all the link pairs, and have excluded SB,R2 and SB,R4 since these routes are not possible. Goods can only move in one direction, from supplier to retailer.

In [53]:
Tuple(links)

(("SA", "R1"), ("SA", "R2"), ("SA", "R3"), ("SA", "R4"), ("SA", "R5"), ("SA", "R6"), ("SA", "R7"), ("SB", "R1"), ("SB", "R3"), ("SB", "R5"), ("SB", "R6"), ("SB", "R7"), ("SC", "R1"), ("SC", "R2"), ("SC", "R3"), ("SC", "R4"), ("SC", "R5"), ("SC", "R6"), ("SC", "R7"))

Next step is to create a dictionary will all the possible links and the cost of moving goods between the links

In [54]:
c_dict = Dict(links .=> c)

Dict{Tuple{String, String}, Int64} with 19 entries:
  ("SA", "R5") => 32
  ("SA", "R6") => 164
  ("SC", "R4") => 26
  ("SB", "R1") => 54
  ("SC", "R7") => 40
  ("SA", "R1") => 78
  ("SB", "R3") => 24
  ("SA", "R4") => 28
  ("SC", "R1") => 48
  ("SC", "R2") => 28
  ("SA", "R7") => 16
  ("SC", "R5") => 56
  ("SC", "R6") => 198
  ("SA", "R2") => 28
  ("SA", "R3") => 22
  ("SB", "R5") => 52
  ("SB", "R6") => 190
  ("SC", "R3") => 34
  ("SB", "R7") => 34

Next we're going to setup the optimization problem.  
The variables will be all the possible route links.  
Our objective is to minimize the cost of moving goods between suppliers and retailers i.e. sum of (number of units moved between each route) * (cost of moving a unit between the route).  
Our constraints are the total number of goods that must be removed from the supplier or moved to the retailer, respectively

In [55]:
using JuMP, GLPK

m = Model(GLPK.Optimizer)

@variable(m, 0<= x[link in links])

@objective(m, Min, sum(c_dict[link] * x[link] for link in links))

count=0

for i in nodes[1:3]
    count += 1
    @constraint(m, sum(x[(k,j)] for (k,j) in links if k == i) == supply[count])
end

count=0

for i in nodes[4:10]
    count += 1
    @constraint(m, sum(x[(k,j)] for (k,j) in links if j == i) == demand[count])
end


In [56]:
print(m)

In [57]:
JuMP.optimize!(m)

In [58]:
println("Objective value: ", JuMP.objective_value(m))

Objective value: 800000.0


In [59]:
for variable in all_variables(m)
    println("$(name(variable)) = $(value(variable))")
end

x[("SA", "R1")] = 0.0
x[("SA", "R2")] = 0.0
x[("SA", "R3")] = 0.0
x[("SA", "R4")] = 0.0
x[("SA", "R5")] = 600.0
x[("SA", "R6")] = 2200.0
x[("SA", "R7")] = 0.0
x[("SB", "R1")] = 0.0
x[("SB", "R3")] = 1200.0
x[("SB", "R5")] = 2000.0
x[("SB", "R6")] = 0.0
x[("SB", "R7")] = 2000.0
x[("SC", "R1")] = 1800.0
x[("SC", "R2")] = 2400.0
x[("SC", "R3")] = 0.0
x[("SC", "R4")] = 800.0
x[("SC", "R5")] = 800.0
x[("SC", "R6")] = 0.0
x[("SC", "R7")] = 0.0


In [60]:
for variable in all_variables(m)
    if value(variable) != 0
        println("$(name(variable)) = $(value(variable))")
    end
end

x[("SA", "R5")] = 600.0
x[("SA", "R6")] = 2200.0
x[("SB", "R3")] = 1200.0
x[("SB", "R5")] = 2000.0
x[("SB", "R7")] = 2000.0
x[("SC", "R1")] = 1800.0
x[("SC", "R2")] = 2400.0
x[("SC", "R4")] = 800.0
x[("SC", "R5")] = 800.0


##### Supplier A will supply:  
600 units to Retailer 5  
200 units to Retailer 6  

##### Supplier B will supply:  
1200 units to Retailer 3  
2000 units to Retailer 5  
2000 units to Retailer 7  

##### Supplier C will supply:  
1800 units to Retailer 1  
2400 units to Retailer 2  
800 units to Retailer 4  
800 units to Retailer 5  

The minimum cost of moving the goods is $800,000.